<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/ice%26fire/Ice%26Fire_import.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install graphdatascience

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 183 kB 13.2 MB/s 
     |████████████████████████████████| 173 kB 62.5 MB/s 
  Created wheel for neo4j: filename=neo4j-5.2.0-py3-none-any.whl size=248021 sha256=b1213438cc3b276397b7ec7db728e7913e3d1e99d4b729b71520a854edd87042
  Stored in directory: /root/.cache/pip/wheels/5a/07/16/4d845d69ef310660c14b7148848c95da3ef3950c7b58daec42
Successfully built neo4j


In [1]:
from graphdatascience import GraphDataScience

host = "bolt://3.231.25.240:7687"
user = "neo4j"
password = "hatchets-visitor-axes"

gds = GraphDataScience(host, auth=(user, password))

# Define constraints

In [2]:
gds.run_cypher(
    """
CREATE CONSTRAINT IF NOT EXISTS FOR (h:Faction) REQUIRE (h.url) IS UNIQUE; 
"""
)

""


In [3]:
gds.run_cypher(
    """
CREATE CONSTRAINT IF NOT EXISTS FOR (c:Character) REQUIRE (c.url) IS UNIQUE; 
"""
)

""


# Import data

In [5]:
gds.run_cypher(
    """
call dbms.setConfigValue('dbms.transaction.timeout','0')
"""
)

""


In [4]:
gds.run_cypher(
    """
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/ice_fire/characters.tsv" AS row FIELDTERMINATOR "\t"
MERGE (c:Character {url: toLower(row.url)})
SET c.name = row.name,
    c.born = replace(replace(replace(row.born, "[", ""), "]", ""),"b'",""),
    c.died = replace(replace(replace(row.died, "[", ""), "]", ""),"b'",""),
    c.title = replace(replace(replace(row.title, "[", ""), "]", ""),"b'","")
FOREACH (a IN apoc.convert.fromJsonList(row.allegiance) | MERGE (f:Faction {url: toLower(split(a, "//")[1])}) MERGE (c)-[:ALLEGIANCE]->(f))
FOREACH (cu IN apoc.convert.fromJsonList(row.culture)   | MERGE (culture:Culture {name: split(toLower(cu), "[")[0]}) MERGE (c)-[:CULTURE]->(culture))
FOREACH (s IN apoc.convert.fromJsonList(row.spouse)     | MERGE (c1:Character {url: toLower(s)}) MERGE (c)-[:SPOUSE]-(c1))
FOREACH (s IN apoc.convert.fromJsonList(row.father)     | MERGE (c1:Character {url: toLower(s)}) MERGE (c)-[:FATHER]->(c1))
FOREACH (m IN apoc.convert.fromJsonList(row.mother)     | MERGE (c1:Character {url: toLower(m)}) MERGE (c)-[:MOTHER]->(c1))
FOREACH (b IN apoc.convert.fromJsonList(row.books)      | MERGE (b1:Book {url: toLower(split(b, "//")[1])}) MERGE (c)-[:APPEARED_IN_BOOK]->(b1))
FOREACH (s IN apoc.convert.fromJsonList(row.show)       | MERGE (s1:Show {url: toLower(split(s, "//")[1])}) MERGE (c)-[:APPEARED_IN_BOOK]->(s1))
FOREACH (pr IN apoc.convert.fromJsonList(row.predecessor) | MERGE (c1:Character {url: toLower(pr)}) MERGE (c)-[:PREDECESSOR]->(c1))
FOREACH (pr IN apoc.convert.fromJsonList(row.successor)   | MERGE (c1:Character {url: toLower(pr)}) MERGE (c)<-[:PREDECESSOR]-(c1))
"""
)

""


# Data cleanup

In [6]:
gds.run_cypher(
    """
MATCH (n)
WHERE NOT n.name IS NOT NULL
WITH n, replace(split(apoc.text.urldecode(n.url), "/")[-1], "_", " ") AS clean_name
SET n.name = clean_name
"""
)

""


In [7]:
gds.run_cypher(
    """
MATCH (t)-[r]->(m)
WHERE toLower(t.name) = toLower(m.name)
DELETE r
RETURN count(*) AS selfloops
"""
)

,selfloops
0,329


In [8]:
gds.run_cypher(
    """
MATCH (n) 
WHERE NOT EXISTS { (n)--() }
DELETE n
RETURN count(*) AS isolated
"""
)

,isolated
0,221


In [9]:
gds.run_cypher(
    """
MATCH (s:Show)
WHERE NOT s.url CONTAINS "house"
WITH s, split(s.url, "_")[-1] AS seasons
WITH seasons, collect(s) AS duplicates
WHERE size(duplicates) > 1
CALL apoc.refactor.mergeNodes(duplicates) YIELD node
RETURN distinct 'done' AS result
"""
)

,result
0,done


In [10]:
gds.run_cypher("""
MATCH (c1:Character), (c2:Character)
WHERE c1.name CONTAINS "catelyn tully" AND c2.name CONTAINS "Catelyn Stark"
CALL apoc.refactor.mergeNodes([c2,c1]) YIELD node
RETURN distinct 'done'
""")

,'done'
0,done


# Verify the data

In [11]:
gds.run_cypher(
    """
CALL apoc.meta.stats()
"""
)

,labelCount,relTypeCount,propertyKeyCount,nodeCount,relCount,labels,relTypes,relTypesCount,stats
0,7,7,86,4319,16941,"{'Character': 3653, 'Book': 20, 'Show': 11, 'C...","{'(:Character)-[:PREDECESSOR]->()': 307, '()-[...","{'APPEARED_IN_BOOK': 7936, 'FATHER': 960, 'MOT...","{'relTypeCount': 7, 'propertyKeyCount': 86, 'l..."


In [12]:
gds.run_cypher(
    """
MATCH (n)
RETURN labels(n)[0] AS label, count(*) AS count
"""
)

,label,count
0,Character,3653
1,Faction,563
2,Culture,72
3,Book,20
4,Show,11
